# For using Cloud

Nhớ restart kernal

In [ ]:
!pip install torch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 --index-url https://download.pytorch.org/whl/cu118

# Import useful library

In [ ]:
from datasets import load_dataset

import torch
from torch import Tensor
import torch.nn as nn
from torch.nn import Transformer
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

import math

# Make dataset

In [ ]:
# Download the dataset via the Hugging Face Datasets library
data = load_dataset("mt_eng_vietnamese", "iwslt2015-en-vi")

In [ ]:
# Tokenization using torchtext basic english tokenizer


# Modelling